## Develop a versatile Q&A chatbot, employing LlamaIndex, ASTRA DB (Apache Cassandra), and Gradient's open-source models like LLama2, all designed for seamless interaction with YouTube videos

[**Link to my YouTube Channel**](https://www.youtube.com/BhaveshBhatt8791?sub_confirmation=1)

Click on the link below to open a Colab version of the notebook. You will be able to create your own version.

<a href="https://colab.research.google.com/github/bhattbhavesh91//youtube-q-a-gradient-astradb/blob/main/youtube-q-a-notebook.ipynb" target="_blank"><img height="40" alt="Run your own notebook in Colab" src = "https://colab.research.google.com/assets/colab-badge.svg"></a>

# Installation

In [ ]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install -q llama-index
!pip install -q tiktoken==0.4.0
!pip install -Uq openai-whisper
!pip install -Uq yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.3/166.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.3/801.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 re

# Imports

In [ ]:
import json
import os
import re
import time
import whisper
import yt_dlp
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index import ServiceContext
from llama_index import set_global_service_context
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings import GradientEmbedding
from llama_index.llms import GradientBaseModelLLM
from llama_index.vector_stores import CassandraVectorStore

# Download Audio from YouTube video function

In [ ]:
def download_audio(link):
    with yt_dlp.YoutubeDL({'extract_audio': True,
                           'format': 'bestaudio',
                           'outtmpl': '%(title)s.mp3'}) as video:
        info_dict = video.extract_info(link, download = True)
        video_title = info_dict['title']
        video.download(link)
    return video_title

# Example to extract audio -

In [ ]:
youtube_video_url = "https://www.youtube.com/watch?v=jBbLon5esbE"

In [ ]:
download_audio(youtube_video_url)

[youtube] Extracting URL: https://www.youtube.com/watch?v=jBbLon5esbE
[youtube] jBbLon5esbE: Downloading webpage
[youtube] jBbLon5esbE: Downloading ios player API JSON
[youtube] jBbLon5esbE: Downloading android player API JSON
[youtube] jBbLon5esbE: Downloading m3u8 information
[info] jBbLon5esbE: Downloading 1 format(s): 251
[download] Destination: Getting Started with Google Cloud Speech-To-Text API Using Python for Novices.mp3
[download] 100% of    3.81MiB in 00:00:00 at 12.51MiB/s  
[youtube] Extracting URL: https://www.youtube.com/watch?v=jBbLon5esbE
[youtube] jBbLon5esbE: Downloading webpage
[youtube] jBbLon5esbE: Downloading ios player API JSON
[youtube] jBbLon5esbE: Downloading android player API JSON
[youtube] jBbLon5esbE: Downloading m3u8 information
[info] jBbLon5esbE: Downloading 1 format(s): 251
[download] Getting Started with Google Cloud Speech-To-Text API Using Python for Novices.mp3 has already been downloaded
[download] 100% of    3.81MiB


'Getting Started with Google Cloud Speech-To-Text API Using Python for Novices'

# Transcribe Audio from mp3 file

In [ ]:
os.makedirs("text_files")

In [ ]:
def transcribe(model, audio):
    result = model.transcribe(audio)
    with open("text_files/transcription.txt", 'w') as f:
        f.write(result["text"])
    return 1

In [ ]:
model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 61.4MiB/s]


In [ ]:
transcribe(model, "Getting Started with Google Cloud Speech-To-Text API Using Python for Novices.mp3")

1

# Setup the DataStax Vector DB Connection

In [ ]:
cloud_config= {
  'secure_connect_bundle': 'secure-connect-bhavesh-astra-test.zip'
}

with open("bhavesh_astra_test-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(132512472427840) ddcc6460-d494-4a85-9fca-fa397e733c54-us-east1.db.astra.datastax.com:29042:31fcfb67-a0cc-46c1-9da4-d3dfe857e08c> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


4.0.11-b86be92b8b5f


# Environment Variables

In [ ]:
os.environ['GRADIENT_ACCESS_TOKEN'] = 'Enter your Gradient Access Token'
os.environ['GRADIENT_WORKSPACE_ID'] = 'Enter your Gradient Workspace ID'

# Define the Gradient's Model Adapter for LLAMA-2

In [ ]:
llm = GradientBaseModelLLM(
    base_model_slug = "llama2-7b-chat",
    max_tokens = 400,
)

# Configure Gradient embeddings

In [ ]:
embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug = "bge-large",
)

# Setup LLAMA Index Service Context

In [ ]:
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size = 256,
)

set_global_service_context(service_context)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Load the Documents

In [ ]:
documents = SimpleDirectoryReader("/content/text_files").load_data()
print(f"Loaded {len(documents)} document(s).")

Loaded 1 document(s).


# Setup and Query Index

In [ ]:
index = VectorStoreIndex.from_documents(documents,
                                        service_context = service_context)
query_engine = index.as_query_engine()

In [ ]:
response_out = query_engine.query("What is used to convert speech to text in the text file?")
print(response_out.response)

The speech to text conversion in the text file is done using Google's speech to text API.


In [ ]:
response_out = query_engine.query("Does this require an API key?")
print(response_out.response)

Yes.

Explanation: In the video, the speaker mentions that in order to use the Google Cloud Speech to Text API, you will require the credentials of the API. The speaker then goes on to explain how to create a JSON file containing the credentials and how to save it to an environment variable called "Google underscore application underscore credentials". This indicates that an API key is required to use the Google Cloud Speech to Text API.


In [ ]:
response_out = query_engine.query("Which programming language was used in the solution?")
print(response_out.response)

Python
